# Summary
In this notebook I am prospecting how to gather chemical and bgc classes from mibigs. This is inspired by ClassifyNPDB.

I will try to use the gnps classyfire api and see how many chemical classifications that results in, so the need for pyclassyfire is obsolete.

In [1]:
import os
import urllib
import glob
import json
from typing import Dict, List
from rdkit import Chem

In [2]:
mibig_folder = "/mnt/scratch/louwe015/mibig_json_2.0/"

base_path = "/mnt/scratch/louwe015/NPLinker/classifying/mibig_classifications/"
mibig_smiles_path = os.path.join(base_path, "files/All_MIBiG_compounds_with_SMILES_and_PMID_MAS.txt")
print("mibig_smiles_path exists is", os.path.isfile(mibig_smiles_path))

mibig_smiles_path exists is True


## Read in mibig smiles

In [16]:
def load_mibig_smiles(in_file: str) -> Dict[str, str]:
    """Load mibig smiles from file (tsv) and return {BGCid_Compound: smiles}
    """
    compound_dict = {}
    lines = 0
    with open(in_file, 'r') as inf:
        inf.readline() #ignore header
        for line in inf:
            lines +=1 
            line = line.split('\t')
            compound_id = line[0]+"_"+line[1]
            structure = line[2]
            if compound_id in compound_dict:
                raise ValueError("Duplication in mibig smiles file, please check file")
            if len(structure) > 0:
                compound_dict[compound_id] = structure
    print(f"Done. Read {lines} lines")
    return compound_dict

In [17]:
comp_dict = load_mibig_smiles(mibig_smiles_path)

Done. Read 2689 lines


In [18]:
list(comp_dict.items())[:5]

[('BGC0000001_abyssomicin C',
  'CC1C[C@]23OC(=O)C4=C2OC1C(O)C3\\C=C/C(=O)[C@@H](C)C[C@@H](C)C4=O'),
 ('BGC0000001_atrop-abyssomicin C',
  'CC1CC23OC(=O)C4=C2OC1C(O)C3\\C=C/C(=O)C(C)CC(C)C4=O'),
 ('BGC0000002_aculeximycin',
  'CCCC(O[C@H]1C[C@](C)(N)[C@H](O)[C@H](C)O1)C(C)C(O)C(CC)\\C=C\\C(O)C(C)C1C\\C=C(C)\\C(O)C(C)C(CC(O)C(C)C(O)CC2CC(O)C(O)C(O)(CC(O[C@@H]3O[C@H](C)[C@@H](O)[C@H](O[C@H]4C[C@@H](N)[C@H](O)[C@@H](C)O4)[C@H]3O[C@@H]3O[C@H](C)[C@@H](O)[C@H](O)[C@H]3O)C(C)CCC(O)CC(O)C\\C=C(CC)\\C(=O)O1)O2)O[C@@H]1O[C@H](CO)[C@@H](O)[C@H](O)[C@@H]1O'),
 ('BGC0000003_AF-toxin',
  'CCC(C)C(C(=O)OC(/C=C/C=C/C=C/C(=O)O)C1(CO1)C)OC(=O)C(C(C)(C)O)O'),
 ('BGC0000004_aflatoxin G1',
  '[H][C@@]12OC=C[C@]1([H])C1=C(O2)C=C(OC)C2=C1OC(=O)C1=C2CCOC1=O')]

In [19]:
len(comp_dict)

2112

## Use GNPS api to get ClassyFire results from smiles
- Try to get it from smiles
- If result is empty; convert to inchikey through gnps as well
- get CF result from inchikey


### First try just using smiles

In [51]:
# just trying with smiles

result_list = []
for i, (np, smiles) in enumerate(comp_dict.items()):
    url_base = "https://gnps-structure.ucsd.edu/classyfire?smiles="
    smiles = smiles.strip(' ')
    safe_smiles = urllib.parse.quote(smiles)  # url encoding
    url = url_base + safe_smiles
    try:
        with urllib.request.urlopen(url) as inf:
            smiles_result = inf.read()
    except urllib.error.HTTPError:
        # apparently the smiles request failed
        smiles_result = None
        superclass = None  # remove later when directing to inchikey

    if smiles_result is not None:
        cf_json = json.loads(smiles_result)  # make a function that gathers result from json, no matter the source
        superclass_dict = cf_json.get("superclass", None)
        superclass = None
        if superclass_dict:
            superclass = superclass_dict.get("name", None)
        
    result_list.append(superclass)

missing_res_num = len([elem for elem in result_list if elem is None])
print(f"Gathered {len(result_list)-missing_res_num} results, missing {missing_res_num} results")

Gathered 1766 results, missing 346 results


### First lookup smiles, then try via lookup of inchikey
apparently this doesn't work.

In [64]:
result_list = []
for i, (np, smiles) in enumerate(comp_dict.items()):
    superclass = None
    
    # lookup CF with smiles
    url_base = "https://gnps-structure.ucsd.edu/classyfire?smiles="
    smiles = smiles.strip(' ')
    safe_smiles = urllib.parse.quote(smiles)  # url encoding
    url = url_base + safe_smiles
    try:
        with urllib.request.urlopen(url) as inf:
            smiles_result = inf.read()
    except urllib.error.HTTPError:
        # apparently the smiles request failed
        smiles_result = None
    
    # read CF result
    if smiles_result is not None:
        cf_json = json.loads(smiles_result)  # make a function that gathers result from json, no matter the source
        superclass_dict = cf_json.get("superclass", None)
        if superclass_dict:
            superclass = superclass_dict.get("name", None)
    
    
    if superclass is None:
        # lookup inchikey from smiles
        url_base = "https://gnps-structure.ucsd.edu/inchikey?smiles="
        url = url_base + safe_smiles
        try:
            with urllib.request.urlopen(url) as inf:
                inchi_result = str(inf.read(), 'utf-8')
#                 print(inchi_result)
        except urllib.error.HTTPError:
            # apparently the inchi request failed
            inchi_result = None
        
        # do CF lookup with inchikey
        if inchi_result is not None:
            url = f"https://gnps-classyfire.ucsd.edu/entities/{inchi_result}.json"
#             print(url)
            try:
                with urllib.request.urlopen(url) as inf:
                    inchi_cf_result = inf.read()
            except urllib.error.HTTPError:
                # apparently the inchi request failed
                inchi_cf_result = None
            
            # read CF result from inchikey lookup
            if inchi_cf_result is not None:
                cf_json = json.loads(inchi_cf_result)  # make a function that gathers result from json, no matter the source
                superclass_dict = cf_json.get("superclass", None)
                if superclass_dict:
                    superclass = superclass_dict.get("name", None)

    result_list.append(superclass)

missing_res_num = len([elem for elem in result_list if elem is None])
print(f"Gathered {len(result_list)-missing_res_num} results, missing {missing_res_num} results")

Gathered 1766 results, missing 346 results


### Try also getting inchikey from rdkit

In [10]:
from rdkit import Chem

result_list = []
for i, (np, smiles) in enumerate(comp_dict.items()):
    superclass = None
    
    # lookup CF with smiles
    url_base = "https://gnps-structure.ucsd.edu/classyfire?smiles="
    smiles = smiles.strip(' ')
    safe_smiles = urllib.parse.quote(smiles)  # url encoding
    url = url_base + safe_smiles
    try:
        with urllib.request.urlopen(url) as inf:
            smiles_result = inf.read()
    except urllib.error.HTTPError:
        # apparently the smiles request failed
        smiles_result = None
    
    # read CF result
    if smiles_result is not None:
        cf_json = json.loads(smiles_result)  # make a function that gathers result from json, no matter the source
        superclass_dict = cf_json.get("superclass", None)
        if superclass_dict:
            superclass = superclass_dict.get("name", None)
    
    if superclass is None:
        # do a second try with inchikey from rdkit
        m = Chem.MolFromSmiles(smiles)
        rdkit_smiles = Chem.MolToSmiles(m, kekuleSmiles=False,isomericSmiles=False)
        m = Chem.MolFromSmiles(rdkit_smiles)
        rdkit_inchi = Chem.inchi.MolToInchiKey(m)
        url = f"https://gnps-classyfire.ucsd.edu/entities/{rdkit_inchi}.json"
        try:
            with urllib.request.urlopen(url) as inf:
                rdkit_inchi_cf_result = inf.read()
        except urllib.error.HTTPError:
            # apparently the inchi request failed
            rdkit_inchi_cf_result = None

        # read CF result from inchikey lookup
        if rdkit_inchi_cf_result is not None:
            cf_json = json.loads(rdkit_inchi_cf_result)  # make a function that gathers result from json, no matter the source
            superclass_dict = cf_json.get("superclass", None)
            if superclass_dict:
                superclass = superclass_dict.get("name", None)

    result_list.append(superclass)

missing_res_num = len([elem for elem in result_list if elem is None])
print(f"Gathered {len(result_list)-missing_res_num} results, missing {missing_res_num} results")

RDKit WARNING: [16:47:06] Conflicting single bond directions around double bond at index 26.
RDKit WARNING: [16:47:06]   BondStereo set to STEREONONE and single bond directions set to NONE.
RDKit ERROR: [16:50:52] Invalid InChI prefix in generating InChI Key


Gathered 1857 results, missing 255 results


## Make some functions
- do_url_request(url: str) -> [bytes, None]
- get_json_cf_results(raw_json: bytes, wanted_info_list: List[str]) -> List[str]
- get inchikey from rdkit
- get_json_npc_results get NPClassifier info from gnps website
- master func

In [6]:
def do_url_request(url: str) -> [bytes, None]:
    """Do url request and return opened .read() object or None if HTTPError is raised
    
    Parameters
    ----------
    url:
        Url to access
    """
    try:
        with urllib.request.urlopen(url) as inf:
            result = inf.read()
    except urllib.error.HTTPError:
        # apparently the request failed
        result = None
    return result

def get_json_cf_results(raw_json: bytes, wanted_keys_list_name: List[str] = ["superclass"]) -> List[str]:
    """Read bytes version of json str, extract the keys in wanted_keys_list_name in order
    
    Parameters
    ----------
    raw_json:
        Json str as a bytes object containing ClassyFire information
    wanted_keys_list_name:
        Keys to extract from the json, they all have a 'name' value in the json
    """
    wanted_info = []
    cf_json = json.loads(raw_json)
    for key in wanted_keys_list_name:
        info_dict = cf_json.get(key, "")
        info = ""
        if info_dict:
            info = info_dict.get('name', "")
        wanted_info.append(info)
    
    return wanted_info

def inchikey_from_smiles_rdkit(smiles: str) -> str:
    """Use rdkit to go from smiles to inchikey
    
    Parameters
    ----------
    smiles:
        Smiles to be turned into inchikey
    """
    m = Chem.MolFromSmiles(smiles)
    smiles = Chem.MolToSmiles(m, kekuleSmiles=False,isomericSmiles=False)
    m = Chem.MolFromSmiles(smiles)
    inchikey = Chem.inchi.MolToInchiKey(m)
    return inchikey

def get_json_npc_results(raw_json: bytes) -> List[str]:
    """Read bytes version of json str, extract the keys in order
    
    Names of the keys extracted in order are:
    class_results, superclass_results, pathway_results, isglycoside.
    List elements are concatonated with '; '.

    Parameters
    ----------
    raw_json:
        Json str as a bytes object containing NPClassifier information
    """
    wanted_info = []
    cf_json = json.loads(raw_json)
    wanted_keys_list = ["class_results", "superclass_results", "pathway_results"]
    last_key = "isglycoside"  # this one returns a bool not a list like the others

    for key in wanted_keys_list:
        info_list = cf_json.get(key, "")
        info = ""
        if info_list:
            info = "; ".join(info_list)
        wanted_info.append(info)

    last_info_bool = cf_json.get(last_key, "")
    last_info = "0"
    if last_info_bool:
        last_info = "1"
    wanted_info.append(last_info)
    
    return wanted_info

In [7]:
wanted_columns_name = ['kingdom', 'superclass', 'class', 'subclass', 'direct_parent']
# ['cf_intermediate_0','cf_intermediate_1',\
#     'cf_intermediate_2','cf_intermediate_3','cf_intermediate_4',\
#     'cf_intermediate_5','cf_molecular_framework','cf_alternative_parents',\
#     'cf_substituents', 'cf_description']

npc_columns_name = ["class_results", "superclass_results", "pathway_results", "isglycoside"]

In [11]:
classes_dict = {}
missed_cfs = 0
missed_npcs = 0
for i, (np, smiles) in enumerate(comp_dict.items()):
    result = None
    npc_result = None
    smiles = smiles.strip(' ')
    rdkit_inchi = inchikey_from_smiles_rdkit(smiles)

    # lookup CF with smiles
    url_base = "https://gnps-structure.ucsd.edu/classyfire?smiles="
    safe_smiles = urllib.parse.quote(smiles)  # url encoding
    url_smiles = url_base + safe_smiles
    smiles_result = do_url_request(url_smiles)
    
    # read CF result
    if smiles_result is not None:
        result = get_json_cf_results(smiles_result, wanted_columns_name)
    
    if not result:
        # do a second try with inchikey from rdkit
        url_inchi = f"https://gnps-classyfire.ucsd.edu/entities/{rdkit_inchi}.json"
        rdkit_inchi_result = do_url_request(url_inchi)

        # read CF result from inchikey lookup
        if rdkit_inchi_result is not None:
            result = get_json_cf_results(rdkit_inchi_result, wanted_columns_name)

    if not result:
        missed_cfs += 1
        result = ['' for _ in range(len(wanted_columns_name))]
    
    # lookup NPClassifier with smiles
    url_base_npc = "https://npclassifier.ucsd.edu/classify?smiles="
    url_smiles_npc = url_base_npc + safe_smiles
    smiles_result_npc = do_url_request(url_smiles_npc)
    
    # read NPC result
    if smiles_result_npc is not None:
        npc_result = get_json_npc_results(smiles_result_npc)
    
    if not npc_result:
        npc_result = ['' for _ in range(len(npc_columns_name))]
    if not npc_result[1]:  # superclass, im assuming this one occurs the most if others are possibly missing
        missed_npcs += 1
    
    # combine results
    combined_result = [smiles, rdkit_inchi] + result + npc_result
    classes_dict[np] = combined_result
    
    assert len(combined_result) == 2+len(wanted_columns_name)+len(npc_columns_name)

print(f"Retrieved {len(classes_dict)-missed_cfs} ClassyFire classes, missing {missed_cfs}")
print(f"Retrieved {len(classes_dict)-missed_npcs} NPClassifier superclasses, missing {missed_npcs}")

RDKit WARNING: [10:48:49] Conflicting single bond directions around double bond at index 26.
RDKit WARNING: [10:48:49]   BondStereo set to STEREONONE and single bond directions set to NONE.
RDKit ERROR: [10:57:17] Invalid InChI prefix in generating InChI Key
RDKit WARNING: [11:09:44] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [11:09:44] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [11:09:44] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [11:09:44] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [11:09:44] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [11:09:44] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [11:09:44] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [11:09:44] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [11:09:46] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [11:09

Retrieved 1859 ClassyFire classes, missing 253
Retrieved 1910 NPClassifier superclasses, missing 202


In [12]:
classes_dict['BGC0000001_abyssomicin C']

['CC1C[C@]23OC(=O)C4=C2OC1C(O)C3\\C=C/C(=O)[C@@H](C)C[C@@H](C)C4=O',
 'FNEADFUPWHAVTA-UHFFFAOYSA-N',
 'Organic compounds',
 'Organoheterocyclic compounds',
 'Oxanes',
 '',
 'Oxanes',
 'Spirotetronate macrolides',
 'Macrolides',
 'Polyketides',
 '0']

In [13]:
header = ['compound_name', "smiles", "inchi_key"] + wanted_columns_name + npc_columns_name
print(header)

['compound_name', 'smiles', 'inchi_key', 'kingdom', 'superclass', 'class', 'subclass', 'direct_parent', 'class_results', 'superclass_results', 'pathway_results', 'isglycoside']


In [14]:
out_base = os.path.split(base_path[:-1])[0]
out_file = os.path.join(out_base, "All_MIBiG_compounds_with_CF_NPC_classes.txt")
print(out_file)
with open(out_file, 'w') as outf:
    outf.write("{}\n".format('\t'.join(header)))
    for name, vals in classes_dict.items():
        outf.write("{}\t{}\n".format(name, '\t'.join(vals)))

/mnt/scratch/louwe015/NPLinker/classifying/All_MIBiG_compounds_with_CF_NPC_classes.txt
